In [26]:
!pip install pytorch-lightning
!pip install lovely-tensors
!pip install lpips
!pip install optuna

In [27]:
import sys
import numpy as np
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset # Added Dataset import
import lovely_tensors as lt
lt.monkey_patch()
from peft import LoraConfig, get_peft_model
import timm
import lpips
from torch.nn import functional as F
from diffusers import ControlNetModel, StableDiffusionControlNetPipeline
import h5py
from torchmetrics import Accuracy
import optuna
import pickle
from tqdm import tqdm
import pandas as pd
import csv
import time
from itertools import product
import glob

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
# Loads ViT-DINOv2 extracted features
train_features_path = "/content/drive/MyDrive/Thesis_KAN/train_features.pt"
val_features_path = "/content/drive/MyDrive/Thesis_KAN/val_features.pt"
test_features_path = "/content/drive/MyDrive/Thesis_KAN/test_features.pt"

In [30]:
num_classes = 2  # binary classification (fire or not)
device = 'cpu'
batch_size = 2048
num_workers = 0  # Set to 0 for Windows compatibility, adjust as needed
ckpt_path = None

In [31]:
interpolation = transforms.InterpolationMode.NEAREST    # if used in transform pipeline

In [48]:
# Check feature shapes
print(torch.load(train_features_path)[0].shape)
print(torch.load(test_features_path)[0].shape)
print(torch.load(val_features_path)[0].shape)

torch.Size([992, 1024])
torch.Size([332, 1024])
torch.Size([330, 1024])


In [33]:
# Load (features, labels) tuple from .pt files
train_features, train_labels = torch.load(train_features_path)
test_features, test_labels = torch.load(test_features_path)

# Print shapes
print("✅ Train features shape:", train_features.shape)
print("✅ Train labels shape  :", train_labels.shape)
print("✅ Test features shape :", test_features.shape)
print("✅ Test labels shape   :", test_labels.shape)

# Print first 5 samples from training set
print("\n🔹 First 5 training features:\n", train_features[:5])
print("\n🔹 First 5 training labels:\n", train_labels[:5])

# Print first 5 samples from test set
print("\n🔸 First 5 test features:\n", test_features[:5])
print("\n🔸 First 5 test labels:\n", test_labels[:5])

✅ Train features shape: torch.Size([992, 1024])
✅ Train labels shape  : torch.Size([992])
✅ Test features shape : torch.Size([332, 1024])
✅ Test labels shape   : torch.Size([332])

🔹 First 5 training features:
 tensor[5, 1024] n=5120 (20Kb) x∈[-5.295, 4.881] μ=-0.003 σ=1.423

🔹 First 5 training labels:
 tensor[5] i64 all_zeros [0, 0, 0, 0, 0]

🔸 First 5 test features:
 tensor[5, 1024] n=5120 (20Kb) x∈[-5.588, 4.551] μ=0.000 σ=1.413

🔸 First 5 test labels:
 tensor[5] i64 all_zeros [0, 0, 0, 0, 0]


In [34]:
!git clone https://github.com/ZiyaoLi/fast-kan.git
!git clone https://github.com/SynodicMonth/ChebyKAN.git
!git clone https://github.com/SpaceLearner/JacobiKAN.git
!git clone https://github.com/sidhu2690/RBF-KAN
!git clone https://github.com/Zhangyanbo/FCN-KAN.git
!git clone https://github.com/GistNoesis/FourierKAN.git
!git clone https://github.com/Blealtan/efficient-kan

fatal: destination path 'fast-kan' already exists and is not an empty directory.
fatal: destination path 'ChebyKAN' already exists and is not an empty directory.
fatal: destination path 'JacobiKAN' already exists and is not an empty directory.
fatal: destination path 'RBF-KAN' already exists and is not an empty directory.
fatal: destination path 'FCN-KAN' already exists and is not an empty directory.
fatal: destination path 'FourierKAN' already exists and is not an empty directory.
fatal: destination path 'efficient-kan' already exists and is not an empty directory.


In [35]:
# Ensures deterministic behavior across runs.
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [36]:
# Define hidden layers via hidden_sizes
class SimpleMLP(nn.Module):
    def __init__(self, input_size=512, hidden_sizes=[512], output_size=512, dropout_rate=0.2):
        super(SimpleMLP, self).__init__()
        self.layers = nn.ModuleList()
        last_size = input_size
        for size in hidden_sizes:
            self.layers.append(nn.Linear(last_size, size))
            self.layers.append(nn.BatchNorm1d(size))
            self.layers.append(nn.SiLU())
            self.layers.append(nn.Dropout(dropout_rate))
            last_size = size
        self.output = nn.Linear(last_size, output_size)

    # Applies all MLP layers + final output layer.
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.output(x)
        return x

In [37]:
# Expects .pt file saved as (features, labels) tuple
class FeatureDataset(Dataset):
    def __init__(self, feature_file):
        self.features, self.labels = torch.load(feature_file)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [82]:
# Builds a two-part model MLP + KAN variant
def get_model(model_name, layer_sizes, latent_size):
    if model_name == 'mlp':
        return SimpleMLP(latent_size, layer_sizes, num_classes), nn.Identity()

    mlp = SimpleMLP(latent_size, [], layer_sizes[0])

#    if model_name == 'efficient-kan':
#        sys.path.insert(0, 'efficient-kan/src/')
#        import efficient_kan
#        kan = efficient_kan.kan.KAN(layers_hidden=[layer_sizes[0], layer_sizes[1], num_classes])

    if model_name == 'efficient-kan':
        sys.path.insert(0, 'efficient-kan/src/')
        from efficient_kan.kan import KAN
        kan = KAN(layers_hidden=[layer_sizes[0], layer_sizes[1], num_classes])

    elif model_name == 'FastKAN':
        sys.path.insert(0, 'fast-kan/fastkan')
        from fastkan import FastKAN
        kan = FastKAN(layers_hidden=[layer_sizes[0], layer_sizes[1], num_classes])

    elif model_name == 'ChebyKAN':
        from ChebyKAN.ChebyKANLayer import ChebyKANLayer
        kan = nn.Sequential(
            nn.LayerNorm(layer_sizes[0]),
            ChebyKANLayer(layer_sizes[0], layer_sizes[1], 4),
            nn.LayerNorm(layer_sizes[1]),
            ChebyKANLayer(layer_sizes[1], num_classes, 4)
        )

    elif model_name == 'JacobiKAN':
        from JacobiKAN.JacobiKANLayer import JacobiKANLayer
        kan = nn.Sequential(
            nn.LayerNorm(layer_sizes[0]),
            JacobiKANLayer(layer_sizes[0], layer_sizes[1], 4),
            nn.LayerNorm(layer_sizes[1]),
            JacobiKANLayer(layer_sizes[1], num_classes, 4)
        )

    elif model_name == 'RBFKAN':
        sys.path.insert(0, 'RBF-KAN')
        from Fast_RBF_KAN import RBFKAN
        kan = RBFKAN([layer_sizes[0], layer_sizes[1], num_classes])

    elif model_name == 'FourierKAN':
        from FourierKAN.fftKAN import NaiveFourierKANLayer
        kan = nn.Sequential(
            nn.SiLU(),
            nn.LayerNorm(layer_sizes[0]),
            NaiveFourierKANLayer(layer_sizes[0], layer_sizes[1], 5),
            nn.SiLU(),
            nn.LayerNorm(layer_sizes[1]),
            NaiveFourierKANLayer(layer_sizes[1], num_classes, 5)
        )

    return mlp, kan

In [83]:
class Lightning_Model(pl.LightningModule):
    def __init__(self, model_name, layer_sizes, latent_size, lr):
        super().__init__()
        self.lr = lr
        self.mlp, self.kan = get_model(model_name, layer_sizes, latent_size)
        self.cross_entropy = nn.CrossEntropyLoss()
        self.train_acc = Accuracy('multiclass', num_classes=num_classes)
        self.val_acc = Accuracy('multiclass', num_classes=num_classes)

    def forward(self, x, batch_idx=None):
        return self.kan(self.mlp(x))

    def training_step(self, batch, batch_idx):
        features, labels = batch
        out = self(features)
        loss = self.cross_entropy(out, labels)
        self.log("train_loss", loss, on_step=True, on_epoch=True)
        self.log("train_acc", self.train_acc(out, labels), on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        features, labels = batch
        out = self(features)
        loss = self.cross_entropy(out, labels)
        self.log("val_cross_entropy", loss, on_step=True, on_epoch=True)
        self.log("val_acc", self.val_acc(out, labels), on_step=True, on_epoch=True)
        return loss

    def configure_optimizers(self):
        return optim.RAdam(self.parameters(), lr=self.lr)

    def train_dataloader(self):
        return DataLoader(FeatureDataset(train_features_path), batch_size=batch_size, shuffle=True, num_workers=num_workers)

    def val_dataloader(self):
        return DataLoader(FeatureDataset(val_features_path), batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [84]:
# Counts number of trainable parameters for comparison.
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [85]:
def extract_latent_size():
    return torch.load(val_features_path)[0].shape[-1]

In [86]:
def objective(params):
    model_name, lr, size1, size2 = params
    layer_sizes = [size1, size2]
    latent_size = extract_latent_size()
    model = Lightning_Model(model_name, layer_sizes, latent_size, lr)
    num_trainable_params = count_parameters(model)

    logger = pl.loggers.CSVLogger(save_dir='logs/', name=model_name)
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        monitor='val_cross_entropy',
        dirpath='checkpoints/',
        filename=f'{model_name}-{{epoch:02d}}-{{val_cross_entropy:.4f}}-{{val_acc:.4f}}',
        save_top_k=1,
        mode='min',
        save_last=True
    )

    trainer = pl.Trainer(
        max_epochs=100,
        accelerator='auto',
        devices=1,
        callbacks=[checkpoint_callback],
        logger=logger,
        check_val_every_n_epoch=33,
        precision='32-true'
    )

    start_time = time.time()
    trainer.fit(model)
    training_time = time.time() - start_time

    # Save training steps
    metrics_path = os.path.join(logger.log_dir, 'metrics.csv')
    detailed_metrics = pd.read_csv(metrics_path)
    detailed_metrics['model_name'] = model_name
    detailed_metrics['layer1'] = size1
    detailed_metrics['layer2'] = size2
    detailed_metrics['lr'] = lr
    os.makedirs('training_steps', exist_ok=True)
    detailed_metrics.to_csv(f'training_steps/{model_name}_steps.csv', index=False)

    val_loss = trainer.callback_metrics.get("val_cross_entropy", float('inf'))
    #best_ckpt_path = checkpoint_callback.best_model_path

    return num_trainable_params, training_time, val_loss

In [87]:
# Main Function Grid Search Over All Models except 'efficient-kan'
def main():
    param_grid = {
        'model_name': ['mlp','efficient-kan','FastKAN', 'ChebyKAN', 'JacobiKAN', 'RBFKAN'],
        'lr': [0.01],
        'n_units_l1': [24],
        'n_units_l2': [16]
    }

    all_params = list(product(param_grid['model_name'], param_grid['lr'], param_grid['n_units_l1'], param_grid['n_units_l2']))
    results = []

    for index, params in tqdm(enumerate(all_params, start=1)):
        num_trainable_params, training_time, val_loss = objective(params)
        results.append((params, num_trainable_params, training_time, val_loss))
        print(f"\nTrial {index} Done: Params={params}, Params={num_trainable_params}, Time={training_time:.2f}s, Loss={val_loss}")

    with open('training_results2.csv', 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['model_name', 'n_units_l1', 'n_units_l2', 'trainable_params', 'time_to_train', 'val_loss'])
        writer.writeheader()
        for params, num_params, train_time, val_loss in results:
            writer.writerow({
                'model_name': params[0],
                'n_units_l1': params[2],
                'n_units_l2': params[3],
                'trainable_params': num_params,
                'time_to_train': train_time,
                'val_loss': val_loss
            })

    step_files = glob.glob("training_steps/*.csv")
    all_steps = pd.concat([pd.read_csv(f) for f in step_files], ignore_index=True)
    all_steps.to_csv("all_training_steps_combined.csv", index=False)

    best_trial = min(results, key=lambda x: x[3])
    print("\nBest trial:")
    print(f"Params: {best_trial[0]}, Loss: {best_trial[3]}")

In [88]:
if __name__ == '__main__':
    main()

0it [00:00, ?it/s]INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:658: Checkpoint directory /content/checkpoints exists and is not empty.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type               | Params | Mode 
-------------------------------------------------------------
0 | mlp           | SimpleMLP          | 25.1 K | train
1 | kan           | Identity           | 0      | train
2 | cross_entropy | CrossEntropyLoss   | 0      | train
3 | train_acc     | MulticlassAccuracy | 0      | train
4 | val_acc       | MulticlassAccuracy | 0      | train
-------------------------------------------------------------
25.1 K    Trainable params
0         Non-trainable params
25.1 K    Total 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
6it [00:44,  7.44s/it]


Trial 6 Done: Params=('RBFKAN', 0.01, 24, 16), Params=28362, Time=5.75s, Loss=0.015317467041313648

Best trial:
Params: ('RBFKAN', 0.01, 24, 16), Loss: 0.015317467041313648


In [89]:
!pip install scikit-learn

In [90]:
def extract_latent_size():
    return torch.load(test_features_path)[0].shape[-1]

In [91]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from torch.utils.data import DataLoader
import torch
import os

def evaluate_model(model_name, model_class, layer_sizes, latent_size, ckpt_path):
    model = model_class(model_name, layer_sizes, latent_size, lr=0.01)
    checkpoint = torch.load(ckpt_path, map_location='cpu')
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()

    test_dataset = FeatureDataset(test_features_path)
    test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False)

    all_preds = []
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for x, y in test_loader:
            logits = model(x)
            probs = torch.softmax(logits, dim=1)[:, 1]  # Probabilities of class 1
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.numpy())
            all_probs.extend(probs.numpy())
            all_labels.extend(y.numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    cm = confusion_matrix(all_labels, all_preds)

    return {
        "model": model_name,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "auc": auc,
        "confusion_matrix": cm
    }

# Evaluate all models
latent_size = extract_latent_size()
layer_sizes = [24, 16]  # Use same as used in training
ckpt_dir = 'checkpoints/'  # Directory where models were saved

model_list = ['mlp', 'efficient-kan', 'FastKAN', 'ChebyKAN', 'JacobiKAN', 'RBFKAN']
results = []

for model_name in model_list:
    # Automatically find best checkpoint file
    ckpts = [f for f in os.listdir(ckpt_dir) if f.startswith(model_name) and f.endswith('.ckpt')]
    if not ckpts:
        print(f"⚠️ No checkpoint found for {model_name}")
        continue
    ckpt_path = os.path.join(ckpt_dir, ckpts[0])  # Assuming first one is best
    metrics = evaluate_model(model_name, Lightning_Model, layer_sizes, latent_size, ckpt_path)
    results.append(metrics)

# Display all results
for r in results:
    print(f"\nModel: {r['model']}")
    print(f"Accuracy: {r['accuracy']:.4f}")
    print(f"Precision: {r['precision']:.4f}")
    print(f"Recall: {r['recall']:.4f}")
    print(f"AUC: {r['auc']:.4f}")
    print(f"Confusion Matrix:\n{r['confusion_matrix']}")



Model: mlp
Accuracy: 0.9880
Precision: 0.9873
Recall: 0.9873
AUC: 0.9971
Confusion Matrix:
[[172   2]
 [  2 156]]

Model: efficient-kan
Accuracy: 0.9910
Precision: 0.9936
Recall: 0.9873
AUC: 0.9958
Confusion Matrix:
[[173   1]
 [  2 156]]

Model: FastKAN
Accuracy: 0.9910
Precision: 1.0000
Recall: 0.9810
AUC: 0.9968
Confusion Matrix:
[[174   0]
 [  3 155]]

Model: ChebyKAN
Accuracy: 0.9940
Precision: 1.0000
Recall: 0.9873
AUC: 0.9940
Confusion Matrix:
[[174   0]
 [  2 156]]

Model: JacobiKAN
Accuracy: 0.9910
Precision: 1.0000
Recall: 0.9810
AUC: 0.9969
Confusion Matrix:
[[174   0]
 [  3 155]]

Model: RBFKAN
Accuracy: 0.9940
Precision: 1.0000
Recall: 0.9873
AUC: 0.9966
Confusion Matrix:
[[174   0]
 [  2 156]]


In [92]:
# prompt: more evaluation matrices for binary class classification

from sklearn.metrics import f1_score, balanced_accuracy_score, average_precision_score

def evaluate_model_extended(model_name, model_class, layer_sizes, latent_size, ckpt_path):
    model = model_class(model_name, layer_sizes, latent_size, lr=0.01)
    checkpoint = torch.load(ckpt_path, map_location='cpu')
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()

    test_dataset = FeatureDataset(test_features_path)
    test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False)

    all_preds = []
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for x, y in test_loader:
            logits = model(x)
            probs = torch.softmax(logits, dim=1)[:, 1]  # Probabilities of class 1
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.numpy())
            all_probs.extend(probs.numpy())
            all_labels.extend(y.numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    cm = confusion_matrix(all_labels, all_preds)

    # Add more metrics
    f1 = f1_score(all_labels, all_preds)
    balanced_accuracy = balanced_accuracy_score(all_labels, all_preds)
    average_precision = average_precision_score(all_labels, all_probs)


    return {
        "model": model_name,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "auc": auc,
        "confusion_matrix": cm,
        "f1_score": f1,
        "balanced_accuracy": balanced_accuracy,
        "average_precision": average_precision,
    }

# Evaluate all models with extended metrics
latent_size = extract_latent_size()
layer_sizes = [24, 16]  # Use same as used in training
ckpt_dir = 'checkpoints/'  # Directory where models were saved

model_list = ['mlp', 'efficient-kan', 'FastKAN', 'ChebyKAN', 'JacobiKAN', 'RBFKAN']
results = []

for model_name in model_list:
    # Automatically find best checkpoint file
    ckpts = [f for f in os.listdir(ckpt_dir) if f.startswith(model_name) and f.endswith('.ckpt')]
    if not ckpts:
        print(f"⚠️ No checkpoint found for {model_name}")
        continue
    ckpt_path = os.path.join(ckpt_dir, ckpts[0])  # Assuming first one is best
    metrics = evaluate_model_extended(model_name, Lightning_Model, layer_sizes, latent_size, ckpt_path)
    results.append(metrics)

# Display all results
for r in results:
    print(f"\nModel: {r['model']}")
    print(f"Accuracy: {r['accuracy']:.4f}")
    print(f"Precision: {r['precision']:.4f}")
    print(f"Recall: {r['recall']:.4f}")
    print(f"AUC: {r['auc']:.4f}")
    print(f"F1 Score: {r['f1_score']:.4f}")
    print(f"Balanced Accuracy: {r['balanced_accuracy']:.4f}")
    print(f"Average Precision: {r['average_precision']:.4f}")
    print(f"Confusion Matrix:\n{r['confusion_matrix']}")



Model: mlp
Accuracy: 0.9880
Precision: 0.9873
Recall: 0.9873
AUC: 0.9971
F1 Score: 0.9873
Balanced Accuracy: 0.9879
Average Precision: 0.9976
Confusion Matrix:
[[172   2]
 [  2 156]]

Model: efficient-kan
Accuracy: 0.9910
Precision: 0.9936
Recall: 0.9873
AUC: 0.9958
F1 Score: 0.9905
Balanced Accuracy: 0.9908
Average Precision: 0.9970
Confusion Matrix:
[[173   1]
 [  2 156]]

Model: FastKAN
Accuracy: 0.9910
Precision: 1.0000
Recall: 0.9810
AUC: 0.9968
F1 Score: 0.9904
Balanced Accuracy: 0.9905
Average Precision: 0.9975
Confusion Matrix:
[[174   0]
 [  3 155]]

Model: ChebyKAN
Accuracy: 0.9940
Precision: 1.0000
Recall: 0.9873
AUC: 0.9940
F1 Score: 0.9936
Balanced Accuracy: 0.9937
Average Precision: 0.9963
Confusion Matrix:
[[174   0]
 [  2 156]]

Model: JacobiKAN
Accuracy: 0.9910
Precision: 1.0000
Recall: 0.9810
AUC: 0.9969
F1 Score: 0.9904
Balanced Accuracy: 0.9905
Average Precision: 0.9974
Confusion Matrix:
[[174   0]
 [  3 155]]

Model: RBFKAN
Accuracy: 0.9940
Precision: 1.0000
Recal

In [95]:
# prompt: models parameter details show in table

# The summary_df already contains the desired evaluation metrics in a table format.
# You can print it directly to display the table.
print("\nModel Performance Summary Table:")
print(summary_df.to_markdown(index=False, floatfmt=".4f"))

# To add the optimizer and trainable parameters, we would need to access the
# `results` list generated in the `main` function which contains this information.
# Since `main` was likely executed before this code block, we can try to load
# the data from the 'training_results2.csv' file saved by the `main` function.

try:
    training_summary_df = pd.read_csv('training_results2.csv')

    # Assuming RAdam was used for all models as defined in Lightning_Model
    training_summary_df['optimizer'] = 'RAdam'

    # Rename columns for clarity
    training_summary_df = training_summary_df.rename(columns={
        'model_name': 'Model',
        'trainable_params': 'Trainable Parameters',
        'time_to_train': 'Training Time (s)',
        'val_loss': 'Loss (Validation)'
    })

    # Merge evaluation results with training results
    # We'll keep the evaluation metrics and add the training ones.
    # Drop redundant columns from training_summary_df that are in summary_df
    training_summary_df_subset = training_summary_df[['Model', 'Loss (Validation)', 'optimizer', 'Trainable Parameters', 'Training Time (s)']]

    # Merge the two dataframes
    combined_summary_df = pd.merge(summary_df, training_summary_df_subset, on='Model', how='left')

    print("\nCombined Model Performance and Training Summary Table:")
    # Exclude confusion matrix and loss if it's already covered by validation loss
    print(combined_summary_df.drop(columns=['confusion_matrix'], errors='ignore').to_markdown(index=False, floatfmt=".4f"))

except FileNotFoundError:
    print("\n'training_results2.csv' not found. Cannot display combined training and evaluation summary.")
    print("Please ensure the `main()` function was executed successfully.")


Model Performance Summary Table:
| Model         |   Accuracy |   Precision |   Recall |    AUC |   F1 Score |   Balanced Accuracy |   Average Precision |
|:--------------|-----------:|------------:|---------:|-------:|-----------:|--------------------:|--------------------:|
| mlp           |     0.9880 |      0.9873 |   0.9873 | 0.9971 |     0.9873 |              0.9879 |              0.9976 |
| efficient-kan |     0.9910 |      0.9936 |   0.9873 | 0.9958 |     0.9905 |              0.9908 |              0.9970 |
| FastKAN       |     0.9910 |      1.0000 |   0.9810 | 0.9968 |     0.9904 |              0.9905 |              0.9975 |
| ChebyKAN      |     0.9940 |      1.0000 |   0.9873 | 0.9940 |     0.9936 |              0.9937 |              0.9963 |
| JacobiKAN     |     0.9910 |      1.0000 |   0.9810 | 0.9969 |     0.9904 |              0.9905 |              0.9974 |
| RBFKAN        |     0.9940 |      1.0000 |   0.9873 | 0.9966 |     0.9936 |              0.9937 |             

In [98]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

# Load extracted features and labels
def load_features_labels(path):
    data = torch.load(path)
    if isinstance(data, (tuple, list)):
        return data[0].numpy(), data[1].numpy()
    return data.numpy()

train_x, train_y = load_features_labels("/content/drive/MyDrive/Thesis_KAN/train_features.pt")
val_x, val_y     = load_features_labels("/content/drive/MyDrive/Thesis_KAN/val_features.pt")
test_x, test_y   = load_features_labels("/content/drive/MyDrive/Thesis_KAN/test_features.pt")

# Optionally combine train + val
X_train = np.concatenate([train_x, val_x], axis=0)
y_train = np.concatenate([train_y, val_y], axis=0)
X_test = test_x
y_test = test_y

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Gradient Boosting": GradientBoostingClassifier(),
    "SVM (Linear)": SVC(kernel='linear', probability=True),
    "SVM (RBF)": SVC(kernel='rbf', probability=True),
    "Naive Bayes": GaussianNB(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Train and evaluate
results = []

for name, model in models.items():
    print(f"Training: {name}")
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    probs = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average='binary')
    rec = recall_score(y_test, preds, average='binary')
    f1 = f1_score(y_test, preds, average='binary')
    cm = confusion_matrix(y_test, preds)
    auc = roc_auc_score(y_test, probs) if probs is not None else "N/A"

    #print(f"{name} -- Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}, AUC: {auc}")
    #print("Confusion Matrix:\n", cm)
    #print("-" * 60)

    results.append({
        "Model": name,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1 Score": f1,
        "AUC": auc,
        "Confusion Matrix": cm
    })


Training: Logistic Regression
Training: Random Forest
Training: Gradient Boosting
Training: SVM (Linear)
Training: SVM (RBF)
Training: Naive Bayes
Training: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:15:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [99]:
import pandas as pd
from tabulate import tabulate

# Prepare results table
summary_table = pd.DataFrame([
    {
        "Model": r["Model"],
        "Accuracy": f"{r['Accuracy']:.4f}",
        "Precision": f"{r['Precision']:.4f}",
        "Recall": f"{r['Recall']:.4f}",
        "F1 Score": f"{r['F1 Score']:.4f}",
        "AUC": f"{r['AUC']:.4f}" if r['AUC'] != "N/A" else "N/A"
    }
    for r in results
])

# Print as formatted table
print(tabulate(summary_table, headers="keys", tablefmt="grid"))

+----+---------------------+------------+-------------+----------+------------+--------+
|    | Model               |   Accuracy |   Precision |   Recall |   F1 Score |    AUC |
+====+=====================+============+=============+==========+============+========+
|  0 | Logistic Regression |     0.991  |      0.9936 |   0.9873 |     0.9905 | 0.9941 |
+----+---------------------+------------+-------------+----------+------------+--------+
|  1 | Random Forest       |     0.9729 |      0.9571 |   0.9873 |     0.972  | 0.9975 |
+----+---------------------+------------+-------------+----------+------------+--------+
|  2 | Gradient Boosting   |     0.9819 |      0.975  |   0.9873 |     0.9811 | 0.9962 |
+----+---------------------+------------+-------------+----------+------------+--------+
|  3 | SVM (Linear)        |     0.991  |      0.9936 |   0.9873 |     0.9905 | 0.9938 |
+----+---------------------+------------+-------------+----------+------------+--------+
|  4 | SVM (RBF)     

In [100]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import math
import torch.nn.functional as F

# Define the KANLinear class
class KANLinear(torch.nn.Module):
    def __init__(
        self,
        in_features,
        out_features,
        grid_size=5,
        spline_order=3,
        scale_noise=0.1,
        scale_base=1.0,
        scale_spline=1.0,
        enable_standalone_scale_spline=True,
        base_activation=torch.nn.SiLU,
        grid_eps=0.02,
        grid_range=[-1, 1],
    ):
        super(KANLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.grid_size = grid_size
        self.spline_order = spline_order

        h = (grid_range[1] - grid_range[0]) / grid_size
        grid = (
            (
                torch.arange(-spline_order, grid_size + spline_order + 1) * h
                + grid_range[0]
            )
            .expand(in_features, -1)
            .contiguous()
        )
        self.register_buffer("grid", grid)

        self.base_weight = torch.nn.Parameter(torch.Tensor(out_features, in_features))
        self.spline_weight = torch.nn.Parameter(
            torch.Tensor(out_features, in_features, grid_size + spline_order)
        )
        if enable_standalone_scale_spline:
            self.spline_scaler = torch.nn.Parameter(
                torch.Tensor(out_features, in_features)
            )

        self.scale_noise = scale_noise
        self.scale_base = scale_base
        self.scale_spline = scale_spline
        self.enable_standalone_scale_spline = enable_standalone_scale_spline
        self.base_activation = base_activation()
        self.grid_eps = grid_eps

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.kaiming_uniform_(self.base_weight, a=math.sqrt(5) * self.scale_base)
        with torch.no_grad():
            noise = (
                (
                    torch.rand(self.grid_size + 1, self.in_features, self.out_features)
                    - 1 / 2
                )
                * self.scale_noise
                / self.grid_size
            )
            self.spline_weight.data.copy_(
                (self.scale_spline if not self.enable_standalone_scale_spline else 1.0)
                * self.curve2coeff(
                    self.grid.T[self.spline_order : -self.spline_order],
                    noise,
                )
            )
            if self.enable_standalone_scale_spline:
                torch.nn.init.kaiming_uniform_(self.spline_scaler, a=math.sqrt(5) * self.scale_spline)

    def b_splines(self, x: torch.Tensor):
        x = x.unsqueeze(-1)
        bases = ((x >= self.grid[:, :-1]) & (x < self.grid[:, 1:])).to(x.dtype)
        for k in range(1, self.spline_order + 1):
            bases = (
                (x - self.grid[:, : -(k + 1)])
                / (self.grid[:, k:-1] - self.grid[:, : -(k + 1)])
                * bases[:, :, :-1]
            ) + (
                (self.grid[:, k + 1 :] - x)
                / (self.grid[:, k + 1 :] - self.grid[:, 1:(-k)])
                * bases[:, :, 1:]
            )
        return bases.contiguous()

    def curve2coeff(self, x: torch.Tensor, y: torch.Tensor):
        A = self.b_splines(x).transpose(0, 1)
        B = y.transpose(0, 1)
        solution = torch.linalg.lstsq(A, B).solution
        result = solution.permute(2, 0, 1)
        return result.contiguous()

    @property
    def scaled_spline_weight(self):
        return self.spline_weight * (
            self.spline_scaler.unsqueeze(-1)
            if self.enable_standalone_scale_spline
            else 1.0
        )

    def forward(self, x: torch.Tensor):
        x = x.reshape(-1, self.in_features)
        base_output = F.linear(self.base_activation(x), self.base_weight)
        spline_output = F.linear(
            self.b_splines(x).view(x.size(0), -1),
            self.scaled_spline_weight.view(self.out_features, -1),
        )
        output = base_output + spline_output
        return output

# Define the KAN class
class KAN(torch.nn.Module):
    def __init__(
        self,
        layers_hidden,
        grid_size=5,
        spline_order=3,
        scale_noise=0.1,
        scale_base=1.0,
        scale_spline=1.0,
        base_activation=torch.nn.SiLU,
        grid_eps=0.02,
        grid_range=[-1, 1],
    ):
        super(KAN, self).__init__()
        self.layers = torch.nn.ModuleList()
        for in_features, out_features in zip(layers_hidden, layers_hidden[1:]):
            self.layers.append(
                KANLinear(
                    in_features,
                    out_features,
                    grid_size=grid_size,
                    spline_order=spline_order,
                    scale_noise=scale_noise,
                    scale_base=scale_base,
                    scale_spline=scale_spline,
                    base_activation=base_activation,
                    grid_eps=grid_eps,
                    grid_range=grid_range,
                )
            )

    def forward(self, x: torch.Tensor):
        for layer in self.layers:
            x = layer(x)
        return x

# Load MNIST dataset and train KAN model
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)
trainset = torchvision.datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
valset = torchvision.datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
valloader = DataLoader(valset, batch_size=64, shuffle=False)

# Define the model, optimizer, scheduler, and loss function
model = KAN([28 * 28, 64, 10])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(10):
    model.train()
    with tqdm(trainloader) as pbar:
        for images, labels in pbar:
            images = images.view(-1, 28 * 28).to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels.to(device))
            loss.backward()
            optimizer.step()
            accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
            pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

    # Validation loop
    model.eval()
    val_loss = 0
    val_accuracy = 0
    with torch.no_grad():
        for images, labels in valloader:
            images = images.view(-1, 28 * 28).to(device)
            output = model(images)
            val_loss += criterion(output, labels.to(device)).item()
            val_accuracy += (output.argmax(dim=1) == labels.to(device)).float().mean().item()
    val_loss /= len(valloader)
    val_accuracy /= len(valloader)

    # Update learning rate
    scheduler.step()

    print(f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}")

100%|██████████| 9.91M/9.91M [00:00<00:00, 35.8MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.62MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 9.29MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 690kB/s]
100%|██████████| 938/938 [01:06<00:00, 14.15it/s, accuracy=0.969, loss=0.12, lr=0.001]


Epoch 1, Val Loss: 0.23441733927080396, Val Accuracy: 0.932921974522293


100%|██████████| 938/938 [00:54<00:00, 17.06it/s, accuracy=0.969, loss=0.105, lr=0.0008]


Epoch 2, Val Loss: 0.16468287180444807, Val Accuracy: 0.9528264331210191


100%|██████████| 938/938 [00:52<00:00, 17.85it/s, accuracy=0.969, loss=0.133, lr=0.00064]


...

Epoch 8, Val Loss: 0.0930439363905543, Val Accuracy: 0.9718351910828026


100%|██████████| 938/938 [00:54<00:00, 17.33it/s, accuracy=1, loss=0.0121, lr=0.000168]


Epoch 9, Val Loss: 0.09015488684337562, Val Accuracy: 0.9740246815286624


100%|██████████| 938/938 [00:54<00:00, 17.26it/s, accuracy=1, loss=0.0125, lr=0.000134]


Epoch 10, Val Loss: 0.09041116127905964, Val Accuracy: 0.972531847133758


In [101]:
# prompt: load a medical dataset and make multiclass classification usjnign all this kan variant and evaluate evaluation metrices

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, f1_score, balanced_accuracy_score, average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from tabulate import tabulate # Need to install tabulate

!pip install tabulate

# Load extracted features and labels
def load_features_labels(path):
    data = torch.load(path)
    if isinstance(data, (tuple, list)):
        return data[0].numpy(), data[1].numpy()
    # Assuming single tensor if not a tuple/list
    return data.numpy()


# Define the extended evaluation function
def evaluate_model_extended(model_name, model_class, layer_sizes, latent_size, ckpt_path, test_features_path):
    """Evaluates a trained model on the test set with extended metrics."""
    model = model_class(model_name, layer_sizes, latent_size, lr=0.01)
    checkpoint = torch.load(ckpt_path, map_location='cpu')
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()

    test_dataset = FeatureDataset(test_features_path)
    test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False)

    all_preds = []
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for x, y in test_loader:
            logits = model(x)
            # Assuming binary classification based on the original code structure
            # If multiclass, adjust probability extraction
            if num_classes == 2:
                 probs = torch.softmax(logits, dim=1)[:, 1]  # Probabilities of class 1
            else:
                 probs = torch.softmax(logits, dim=1) # Probabilities for each class

            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.numpy())
            if num_classes == 2:
                all_probs.extend(probs.numpy())
            else:
                all_probs.extend(probs.numpy().tolist()) # Store as list of lists/arrays
            all_labels.extend(y.numpy())

    # Calculate metrics based on num_classes
    if num_classes == 2:
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary')
        recall = recall_score(all_labels, all_preds, average='binary')
        auc = roc_auc_score(all_labels, all_probs)
        f1 = f1_score(all_labels, all_preds, average='binary')
        balanced_accuracy = balanced_accuracy_score(all_labels, all_preds)
        average_precision = average_precision_score(all_labels, all_probs)
    else:
        # For multiclass, use appropriate average methods
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='weighted') # or 'macro', 'micro'
        recall = recall_score(all_labels, all_preds, average='weighted') # or 'macro', 'micro'
        # AUC for multiclass is more complex, requires 'ovr' or 'ovo' strategy
        # Let's calculate AUC for each class vs rest (OvR)
        try:
            auc = roc_auc_score(all_labels, np.array(all_probs), multi_class='ovr', average='weighted')
        except ValueError as e:
            print(f"Could not calculate AUC for multiclass: {e}")
            auc = "N/A"

        f1 = f1_score(all_labels, all_preds, average='weighted') # or 'macro', 'micro'
        balanced_accuracy = balanced_accuracy_score(all_labels, all_preds)
        # Average Precision for multiclass is also more complex
        try:
            average_precision = average_precision_score(all_labels, np.array(all_probs), average='weighted')
        except ValueError as e:
            print(f"Could not calculate Average Precision for multiclass: {e}")
            average_precision = "N/A"


    cm = confusion_matrix(all_labels, all_preds)


    return {
        "model": model_name,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "auc": auc,
        "confusion_matrix": cm,
        "f1_score": f1,
        "balanced_accuracy": balanced_accuracy,
        "average_precision": average_precision,
    }

# --- Medical Dataset Loading and Preprocessing ---

# Assuming your medical dataset is stored in the specified .pt files containing features and labels
# The existing code already loads these features and labels:
# train_features, train_labels = torch.load(train_features_path)
# test_features, test_labels = torch.load(test_features_path)
# val_features, val_labels = torch.load(val_features_path) # Need val labels too for evaluation if using val_features
# For evaluation, we need the labels for the validation set if it's used for evaluation metrics during training.
# However, the evaluate_model_extended function uses the test_features_path.
# Make sure num_classes is set correctly based on your medical dataset.
# Example: If you have 3 classes (e.g., Healthy, Mild, Severe), set num_classes = 3
# num_classes = 3 # Example for a 3-class problem

# Check if num_classes is already defined and update if needed based on your dataset
# You might need to inspect your labels to determine the actual number of classes
if 'num_classes' not in locals() or num_classes == 2:
    # Infer num_classes from labels if not already set or set to 2
    all_labels_combined = np.concatenate([train_labels.numpy(), test_labels.numpy()])
    unique_labels = np.unique(all_labels_combined)
    num_classes = len(unique_labels)
    print(f"Inferred num_classes from labels: {num_classes}")

# --- Evaluation of KAN Variants and MLP ---

latent_size = extract_latent_size()
layer_sizes = [24, 16]  # Use same as used in training
ckpt_dir = 'checkpoints/'  # Directory where models were saved

# Ensure the required KAN classes are imported or defined
# Based on your preceding code, these should be available
# from efficient_kan.kan import KAN
# from fastkan import FastKAN
# from ChebyKAN.ChebyKANLayer import ChebyKANLayer
# from JacobiKAN.JacobiKANLayer import JacobiKANLayer
# from Fast_RBF_KAN import RBFKAN
# from FourierKAN.fftKAN import NaiveFourierKANLayer


model_list = ['mlp', 'efficient-kan', 'FastKAN', 'ChebyKAN', 'JacobiKAN', 'RBFKAN']
results_kan_variants = []

print("\nEvaluating KAN variants and MLP...")
for model_name in model_list:
    # Automatically find best checkpoint file by validation loss
    ckpts = glob.glob(os.path.join(ckpt_dir, f'{model_name}-*.ckpt'))
    if not ckpts:
        print(f"⚠️ No checkpoint found for {model_name} in {ckpt_dir}")
        continue

    # Sort checkpoints to find the best based on validation loss (assuming filename format)
    # The filename format is {model_name}-{epoch:02d}-{val_cross_entropy:.4f}-{val_acc:.4f}
    # We need to parse the validation loss from the filename to find the minimum
    def parse_val_loss_from_ckpt(ckpt_path):
        try:
            filename = os.path.basename(ckpt_path)
            parts = filename.split('-')
            # Find the part containing the validation loss
            loss_part = [p for p in parts if '.' in p and 'val_cross_entropy' not in p and 'val_acc' not in p]
            if loss_part:
                return float(loss_part[-2]) # Assuming the second to last float is the loss
            return float('inf') # Return infinity if loss cannot be parsed
        except Exception as e:
            print(f"Error parsing loss from filename {ckpt_path}: {e}")
            return float('inf')

    best_ckpt_path = min(ckpts, key=parse_val_loss_from_ckpt)

    print(f"Evaluating {model_name} using checkpoint: {best_ckpt_path}")

    try:
        metrics = evaluate_model_extended(model_name, Lightning_Model, layer_sizes, latent_size, best_ckpt_path, test_features_path)
        results_kan_variants.append(metrics)
    except Exception as e:
        print(f"Error evaluating {model_name} from {best_ckpt_path}: {e}")


# Display results for KAN variants and MLP
print("\n--- KAN Variants and MLP Evaluation Results ---")
summary_data_kan = []
for r in results_kan_variants:
    summary_data_kan.append({
        "Model": r["model"],
        "Accuracy": r["accuracy"],
        "Precision": r["precision"],
        "Recall": r["recall"],
        "F1 Score": r["f1_score"],
        "Balanced Accuracy": r["balanced_accuracy"],
        "Average Precision": r["average_precision"],
        "AUC": r["auc"],
        # Confusion Matrix is typically printed separately or visualized
    })

summary_df_kan = pd.DataFrame(summary_data_kan)

# Add training information if available
try:
    training_summary_df = pd.read_csv('training_results2.csv')

    # Assuming RAdam was used for all models
    training_summary_df['optimizer'] = 'RAdam'

    training_summary_df = training_summary_df.rename(columns={
        'model_name': 'Model',
        'trainable_params': 'Trainable Parameters',
        'time_to_train': 'Training Time (s)',
        'val_loss': 'Loss (Validation)'
    })

    training_summary_df_subset = training_summary_df[['Model', 'Loss (Validation)', 'optimizer', 'Trainable Parameters', 'Training Time (s)']]

    combined_summary_df_kan = pd.merge(summary_df_kan, training_summary_df_subset, on='Model', how='left')

    print("\nCombined KAN Variants and MLP Performance and Training Summary Table:")
    print(combined_summary_df_kan.to_markdown(index=False, floatfmt=".4f"))

    # Display Confusion Matrices for KAN variants and MLP
    print("\nConfusion Matrices for KAN Variants and MLP:")
    for r in results_kan_variants:
        print(f"\nModel: {r['model']}")
        print("Confusion Matrix:")
        print(r['confusion_matrix'])


except FileNotFoundError:
    print("\n'training_results2.csv' not found. Cannot display combined training and evaluation summary for KAN variants and MLP.")
    print("Please ensure the `main()` function was executed successfully to generate this file.")

# --- Evaluation of Traditional ML Models ---

# Assuming you want to run the traditional ML models as a baseline comparison
print("\nEvaluating Traditional Machine Learning Models...")

# Load and preprocess data for traditional ML models
# Ensure test_features_path and other paths are correct
train_x, train_y = load_features_labels(train_features_path)
val_x, val_y     = load_features_labels(val_features_path)
test_x, test_y   = load_features_labels(test_features_path)

# Optionally combine train + val
X_train = np.concatenate([train_x, val_x], axis=0)
y_train = np.concatenate([train_y, val_y], axis=0)
X_test = test_x
y_test = test_y

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define traditional models - ensure num_classes is handled if it's multiclass
models_traditional = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Gradient Boosting": GradientBoostingClassifier(),
    # SVC probability=True can be slow on large datasets
    "SVM (Linear)": SVC(kernel='linear', probability=True),
    "SVM (RBF)": SVC(kernel='rbf', probability=True),
    "Naive Bayes": GaussianNB(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss' if num_classes==2 else 'mlogloss') # Adjust eval_metric for multiclass
}

# Train and evaluate traditional models
results_traditional = []

for name, model in models_traditional.items():
    print(f"Training and Evaluating: {name}")
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    if hasattr(model, "predict_proba"):
        probs = model.predict_proba(X_test)
        if num_classes == 2:
            probs_for_auc = probs[:, 1] # Probability of the positive class
        else:
            probs_for_auc = probs # Probabilities for each class
    else:
        probs = None
        probs_for_auc = None # Cannot calculate AUC without probabilities

    acc = accuracy_score(y_test, preds)
    cm = confusion_matrix(y_test, preds)

    # Calculate metrics based on num_classes
    if num_classes == 2:
        prec = precision_score(y_test, preds, average='binary')
        rec = recall_score(y_test, preds, average='binary')
        f1 = f1_score(y_test, preds, average='binary')
        balanced_acc = balanced_accuracy_score(y_test, preds)
        avg_prec = average_precision_score(y_test, probs_for_auc) if probs_for_auc is not None else "N/A"
        auc = roc_auc_score(y_test, probs_for_auc) if probs_for_auc is not None else "N/A"
    else:
        prec = precision_score(y_test, preds, average='weighted') # or 'macro', 'micro'
        rec = recall_score(y_test, preds, average='weighted') # or 'macro', 'micro'
        f1 = f1_score(y_test, preds, average='weighted') # or 'macro', 'micro'
        balanced_acc = balanced_accuracy_score(y_test, preds)
        # Multiclass Avg Precision
        try:
             avg_prec = average_precision_score(y_test, probs_for_auc, average='weighted') if probs_for_auc is not None else "N/A"
        except ValueError as e:
            print(f"Could not calculate Average Precision for multiclass ({name}): {e}")
            avg_prec = "N/A"
        # Multiclass AUC (OvR)
        try:
            auc = roc_auc_score(y_test, probs_for_auc, multi_class='ovr', average='weighted') if probs_for_auc is not None else "N/A"
        except ValueError as e:
            print(f"Could not calculate AUC for multiclass ({name}): {e}")
            auc = "N/A"


    results_traditional.append({
        "Model": name,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1 Score": f1,
        "Balanced Accuracy": balanced_acc,
        "Average Precision": avg_prec,
        "AUC": auc,
        "Confusion Matrix": cm
    })

# Prepare and display traditional ML results table
print("\n--- Traditional Machine Learning Model Evaluation Results ---")
summary_data_traditional = []
for r in results_traditional:
    summary_data_traditional.append({
        "Model": r["Model"],
        "Accuracy": f"{r['Accuracy']:.4f}",
        "Precision": f"{r['Precision']:.4f}",
        "Recall": f"{r['Recall']:.4f}",
        "F1 Score": f"{r['F1 Score']:.4f}",
        "Balanced Accuracy": f"{r['Balanced Accuracy']:.4f}",
        "Average Precision": f"{r['Average Precision']:.4f}" if r['Average Precision'] != "N/A" else "N/A",
        "AUC": f"{r['AUC']:.4f}" if r['AUC'] != "N/A" else "N/A",
    })

summary_df_traditional = pd.DataFrame(summary_data_traditional)
print(tabulate(summary_df_traditional, headers="keys", tablefmt="grid"))

# Display Confusion Matrices for Traditional ML models
print("\nConfusion Matrices for Traditional Machine Learning Models:")
for r in results_traditional:
    print(f"\nModel: {r['Model']}")
    print("Confusion Matrix:")
    print(r['Confusion Matrix'])

# --- Conclusion / Comparison ---
print("\n--- Overall Comparison ---")
# You can now visually compare the two tables (KAN/MLP vs Traditional ML)
# or combine them into one large table if desired.

# Example of combining and sorting by a metric like Accuracy
# combined_results_all = results_kan_variants + results_traditional
# combined_summary_df_all = pd.DataFrame([
#     {
#         "Model": r["model"] if "model" in r else r["Model"], # Handle different keys
#         "Accuracy": r["accuracy"] if "accuracy" in r else r["Accuracy"],
#         "Precision": r["precision"] if "precision" in r else r["Precision"],
#         "Recall": r["recall"] if "recall" in r else r["Recall"],
#         "F1 Score": r["f1_score"] if "f1_score" in r else r["F1 Score"],
#         "Balanced Accuracy": r["balanced_accuracy"] if "balanced_accuracy" in r else r["Balanced Accuracy"],
#         "Average Precision": r["average_precision"] if "average_precision" in r else r["Average Precision"],
#         "AUC": r["auc"] if "auc" in r else r["AUC"],
#         # Add trainable params and training time if available for all models
#     }
#     for r in combined_results_all
# ])
# combined_summary_df_all = combined_summary_df_all.sort_values(by='Accuracy', ascending=False)
# print("\nCombined Summary of All Models (Sorted by Accuracy):")
# print(tabulate(combined_summary_df_all, headers="keys", tablefmt="grid"))


Inferred num_classes from labels: 2

Evaluating KAN variants and MLP...
Evaluating mlp using checkpoint: checkpoints/mlp-epoch=98-val_cross_entropy=0.0446-val_acc=0.9970.ckpt
Evaluating efficient-kan using checkpoint: checkpoints/efficient-kan-epoch=98-val_cross_entropy=0.0174-val_acc=0.9909.ckpt
Evaluating FastKAN using checkpoint: checkpoints/FastKAN-epoch=98-val_cross_entropy=0.0230-val_acc=0.9939.ckpt
Evaluating ChebyKAN using checkpoint: checkpoints/ChebyKAN-epoch=98-val_cross_entropy=0.0447-val_acc=0.9940.ckpt
Evaluating JacobiKAN using checkpoint: checkpoints/JacobiKAN-epoch=98-val_cross_entropy=0.0196-val_acc=0.9939.ckpt
Evaluating RBFKAN using checkpoint: checkpoints/RBFKAN-epoch=98-val_cross_entropy=0.0460-val_acc=0.9940.ckpt

--- KAN Variants and MLP Evaluation Results ---

Combined KAN Variants and MLP Performance and Training Summary Table:
| Model         |   Accuracy |   Precision |   Recall |   F1 Score |   Balanced Accuracy |   Average Precision |    AUC | Loss (Valida

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:57:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



--- Traditional Machine Learning Model Evaluation Results ---
+----+---------------------+------------+-------------+----------+------------+---------------------+---------------------+--------+
|    | Model               |   Accuracy |   Precision |   Recall |   F1 Score |   Balanced Accuracy |   Average Precision |    AUC |
+====+=====================+============+=============+==========+============+=====================+=====================+========+
|  0 | Logistic Regression |     0.991  |      0.9936 |   0.9873 |     0.9905 |              0.9908 |              0.9966 | 0.9941 |
+----+---------------------+------------+-------------+----------+------------+---------------------+---------------------+--------+
|  1 | Random Forest       |     0.9729 |      0.9571 |   0.9873 |     0.972  |              0.9736 |              0.997  | 0.9971 |
+----+---------------------+------------+-------------+----------+------------+---------------------+---------------------+--------+
|  2 |

# Image to feature extractor